# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
normed_mpia_authors = [k[1] for k in mpia_authors]   # initials + fullname
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [mpia.get_initials(k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, normed_mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

J. Li  ->  J. Li  |  ['J. Li']
H. Beuther  ->  H. Beuther  |  ['H. Beuther']
L. Boogaard  ->  L. Boogaard  |  ['L. Boogaard']
M. Flock  ->  M. Flock  |  ['M. Flock']
K. Kreckel  ->  K. Kreckel  |  ['K. Kreckel']
T. Ueda  ->  T. Ueda  |  ['T. Ueda']
Arxiv has 50 new papers today
          2 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [4]:
documents = []
failed = []
for paper in tqdm(candidates):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [mpia.get_initials(k) for k in doc.authors], 
                normed_mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/2 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2311.09752


extracting tarball to tmp_2311.09752...

 done.


J. Li  ->  J. Li  |  ['J. Li']


Found 72 bibliographic references in tmp_2311.09752/sample631.bbl.
Retrieving document from  https://arxiv.org/e-print/2311.10056


extracting tarball to tmp_2311.10056...

 done.


/opt/hostedtoolcache/Python/3.9.18/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3548: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.18/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3548: LatexWarning: Found documentclass in tmp_2311.10056/main.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.18/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:399: LatexWarning: Latex injecting: 'Contributions/zhao01' from 'tmp_2311.10056/Contributions/zhao01.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.9.18/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:399: LatexWarning: Latex injecting: 'Contributions/xu01' from 'tmp_2311.10056/Contributions/xu01.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.9.18/x64/

### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [5]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2311.09752-b31b1b.svg)](https://arxiv.org/abs/arXiv:2311.09752) | **The Distribution of Semi-Detached Binaries. I.An Efficient Pipeline**  |
|| J. Xiong, et al. -- incl., <mark>J. Li</mark> |
|*Appeared on*| *2023-11-17*|
|*Comments*| **|
|**Abstract**| Semi-detached binaries are in the stage of mass transfer and play a crucial role in studying mass transfer physics between interacting binaries. Large-scale time-domain surveys provide massive light curves of binary systems, while Gaia offers high-precision astrometric data. In this paper, we develop, validate, and apply a pipeline that combines the MCMC method with a forward model and DBSCAN clustering to search for semi-detached binary and estimate its inclination, relative radius, mass ratio, and temperature ratio using light curve. We train our model on the mock light curves from PHOEBE, which provides broad coverage of light curve simulations for semi-detached binaries. Applying our pipeline to TESS sectors 1-26, we have identified 77 semi-detached binary candidates. Utilizing the distance from Gaia, we determine their masses and radii with median fractional uncertainties of ~26% and ~7%, respectively. With the added 77 candidates, the catalog of semi-detached binaries with orbital parameters has been expanded by approximately 20%. The comparison and statistical results show that our semi-detached binary candidates align well with the compiled samples and the PARSEC model in Teff-L and M-R relations. Combined with the literature samples, comparative analysis with stability criteria for conserved mass transfer indicates that ~97.4% of samples are undergoing nuclear-timescale mass transfer, and two samples (GO Cyg and TIC 454222105) are located within the limits of stability criteria for dynamical- and thermal-timescale mass transfer, which are currently undergoing thermal-timescale mass transfer. Additionally, one system (IR Lyn) is very close to the upper limit of delayed dynamical-timescale mass transfer. |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2311.10056-b31b1b.svg)](https://arxiv.org/abs/arXiv:2311.10056) | **A Collection of German Science Interests in the Next Generation Very  Large Array**  |
|| M. Kadler, et al. -- incl., <mark>H. Beuther</mark>, <mark>L. Boogaard</mark>, <mark>M. Flock</mark>, <mark>K. Kreckel</mark>, <mark>T. Ueda</mark> |
|*Appeared on*| *2023-11-17*|
|*Comments*| *Version 1.0 (status November 16, 2023): 143 pages, comments and future contributions welcome*|
|**Abstract**| The Next Generation Very Large Array (ngVLA) is a planned radio interferometer providing unprecedented sensitivity at wavelengths between 21 cm and 3 mm. Its 263 antenna element array will be spatially distributed across North America to enable both superb low surface brightness recovery and sub-milliarcsecond angular resolution imaging. The project was developed by the international astronomy community under the lead of the National Radio Astronomy Observatory (NRAO), and is anticipated to be built between 2027 and 2037. Two workshops have been held in 2022 and 2023 with the goal to discuss and consolidate the scientific interests in the ngVLA within the German astronomical community. This community paper constitutes a collection of 41 science ideas which the German community aims to pursue with the ngVLA in the 2030s. This is not a complete list and the ideas are not developed at the level of a "Science Book", such that the present document is mainly to be considered a "living document", to provide a basis for further discussion within the community. As such, additional contributions are welcome, and will be considered for inclusion in future revisions. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: '69117' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [6]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [7]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2311.09752.md
    + _build/html/tmp_2311.09752/./figure3a.png
    + _build/html/tmp_2311.09752/./figure3b.png
    + _build/html/tmp_2311.09752/./figure3c.png
    + _build/html/tmp_2311.09752/./figure3d.png
    + _build/html/tmp_2311.09752/./figure9a.png
    + _build/html/tmp_2311.09752/./figure9b.png
    + _build/html/tmp_2311.09752/./figure9c.png
    + _build/html/tmp_2311.09752/./figure9d.png


## Display the papers

Not necessary but allows for a quick check.

In [8]:
[display(Markdown(k[1])) for k in documents];

, including the ZAMS in dashed lines, the LHG (late Hertzsprung–Russell gap) in dotted lines, and the BRGB (base of the red giant branch) in solid lines. The red star markers represent our semi-detached binary candidates, while the black dots correspond to the literature samples. (*fig:PQ*)

</div>
<div id="div_fig2">

<img src="tmp_2311.09752/./figure3a.png" alt="Fig3.1" width="25%"/><img src="tmp_2311.09752/./figure3b.png" alt="Fig3.2" width="25%"/><img src="tmp_2311.09752/./figure3c.png" alt="Fig3.3" width="25%"/><img src="tmp_2311.09752/./figure3d.png" alt="Fig3.4" width="25%"/>

**Figure 3. -** The upper plots display the results of the model with a more massive component filling its Roche lobe, while the bottom panels show the results of the model with a less massive component filling its Roche lobe. Panels (a) and (c) are cumulative histograms for standard deviations of the residuals ($\sigma\rm_{Res}$) between the predicted light curves and the simulated light curves. Panel (b) shows a direct comparison between the predicted light curve (red solid line) and the true light curve (black dots) for a target with the more massive component filling its Roche lobe, and the parameters are $T\rm_{1}$= 37768 K, $i$=68.97$^{\circ}$, $R\rm_{2}/a$=0.265, $q$=0.924, and $T\rm_{2}$/$T\rm_{1}$=0.892. Similarly, panel (d) presents a direct comparison for a target with the less massive component filling its Roche lobe, with the parameters being $T\rm_{1}$= 29179 K, $i$=89.90$^{\circ}$, $R\rm_{1}/a$=0.235, $q$=0.756, and $T\rm_{2}$/$T\rm_{1}$=1.118. In panels (b) and (d), the corresponding residuals between predictions and true values are shown at the bottom. (*fig:model_precision*)

</div>
<div id="div_fig3">

<img src="tmp_2311.09752/./figure9a.png" alt="Fig9.1" width="25%"/><img src="tmp_2311.09752/./figure9b.png" alt="Fig9.2" width="25%"/><img src="tmp_2311.09752/./figure9c.png" alt="Fig9.3" width="25%"/><img src="tmp_2311.09752/./figure9d.png" alt="Fig9.4" width="25%"/>

**Figure 9. -** The distribution of relative uncertainties ($\sigma_{M}/M$ and $\sigma_{R}/R$) on masses ((a) and (b)) and radii ((c)and (d)) for our candidates that derived from Gaia three bands by using the parameters from Gaia golden sample. The black dotted, blue dashed, and red solid represent the relative uncertainties for Gaia’s
G-, BP- and RP-band, respectively. (*fig:MR_sec_precision*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2311.09752"></div>

# Create HTML index

In [9]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

322  publications files modified in the last 7 days.


In [10]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

2  publications in the last 7 days.


In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [13]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

2  publications in the last day.


In [14]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
